<a href="https://colab.research.google.com/github/SolNam-UI/CTD/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import부

In [1]:
!pip install quickdraw tensorflow ColabTurtle Pillow

  Preparing metadata (setup.py) ... done
  Created wheel for ColabTurtle: filename=ColabTurtle-2.1.0-py3-none-any.whl size=7643 sha256=6d748558253b460da3eea15dd76df3ea14665b56cdf45bc741259e61d75bde85
  Stored in directory: /root/.cache/pip/wheels/f6/9e/81/137e7da25129474562d30f8660be599e5c8d79228cb747e5b9
Successfully built ColabTurtle


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/파이썬 프로젝트 CTD"
%run 8409b3feec20f159d8a50b0a811d3bca/draw.py #Colab 환경에서 사용자가 그릴 수 있게 해주는 오픈소스
%cd "/content/drive/Mydrive/파이썬 프로젝트 CTD/storage"

Mounted at /content/drive
/content/drive/MyDrive/파이썬 프로젝트 CTD
[Errno 2] No such file or directory: '/content/drive/Mydrive/파이썬 프로젝트 CTD/storage'
/content/drive/MyDrive/파이썬 프로젝트 CTD


In [28]:
import numpy as np
import random
import PIL.Image
from PIL import Image
from quickdraw import QuickDrawDataGroup, QuickDrawData
from keras import layers
from keras import models
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from ColabTurtle import *

Main 프로그램부

In [31]:
# 그림 그리기
def inputDrawing():
    draw(filename="dd.png", w=280, h=280, line_width=10)

    # 사용자가 그린 이미지 전처리
    img = Image.open("/content/drive/MyDrive/파이썬 프로젝트 CTD/dd.png").resize((28, 28)).convert("L")
    imgTemp = np.array(img) / 255.0
    inData = imgTemp.reshape(1, 28, 28, 1)

    predict(inData)

def predict(inData):
    # 예측
    prediction = model.predict(inData)
    result = trainList[np.argmax(prediction)]
    print("정답은: ", result + "입니다.")
    if (result == goalImg) :
        print("성공!")
    else:
        print("실패!")

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 정의
model = models.Sequential()
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(50, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# 이미지 리스트 로드
trainList = QuickDrawData().drawing_names[:50]

# 이미지 학습
trainImg, trainLabel = [], []
for i, label in enumerate(trainList):
    drawings = list(QuickDrawDataGroup(label).drawings)[:100]
    for d in drawings:
        img = d.image.resize((28, 28)).convert("L")
        img_array = np.array(img) / 255.0
        trainImg.append(img_array)
        trainLabel.append(i)

trainImg = np.array(trainImg)
trainImg = trainImg.reshape(-1, 28, 28, 1)

trainLabel = np.array(trainLabel)

# 모델 학습
model.fit(trainImg, trainLabel, epochs=10, batch_size=32, validation_split=0.2, callbacks=[reduce_lr, early_stop])

# 이미지 리스트에서 그릴 이미지 할당
goalImg = trainList[int(random.randrange(0,49))]
print("목표 그림: ", goalImg)

inputDrawing()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


loading aircraft carrier drawings
load complete
loading airplane drawings
load complete
loading alarm clock drawings
load complete
loading ambulance drawings
load complete
loading angel drawings
load complete
loading animal migration drawings
load complete
loading ant drawings
load complete
loading anvil drawings
load complete
loading apple drawings
load complete
loading arm drawings
load complete
loading asparagus drawings
load complete
loading axe drawings
load complete
loading backpack drawings
load complete
loading banana drawings
load complete
loading bandage drawings
load complete
loading barn drawings
load complete
loading baseball bat drawings
load complete
loading baseball drawings
load complete
loading basket drawings
load complete
loading basketball drawings
load complete
loading bat drawings
load complete
loading bathtub drawings
load complete
loading beach drawings
load complete
loading bear drawings
load complete
loading beard drawings
load complete
loading bed drawings
l

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
정답은:  blackberry입니다.
실패!
